In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.models import load_model

2025-08-10 01:11:44.193915: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-10 01:11:44.267589: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-10 01:11:44.267647: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-10 01:11:44.269462: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-10 01:11:44.281287: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-10 01:11:44.282414: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [7]:
model=load_model('model.h5')

with open('label_encoder_gender.pkl', 'rb')as file:
    label_encoder_gender=pickle.load(file)
    
with open('ohe_geo.pkl','rb')as file:
    ohe_geo=pickle.load(file)
    
with open('scaler.pkl','rb')as file:
    scaler=pickle.load(file)

In [9]:
input_data={
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age' : 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
    
}

In [10]:
geo_encoded=ohe_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded, columns=ohe_geo.get_feature_names_out(['Geography']))
geo_encoded_df

/home/fida/Music/ANN CLASSIFICATION/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [12]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [14]:
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [15]:
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [16]:
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [17]:
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 76ms/step


array([[0.03904787]], dtype=float32)

In [19]:
prediction_proba=prediction[0][0]
prediction_proba

0.039047875

In [20]:
if prediction_proba > 0.5:
    print("the customer is likely to churn")
else:
    print('the customer is not most likely to churn')

the customer is not most likely to churn
